# 📐 Vorlesung 07 - Lineare Regression und Least Squares

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_07/vorlesung_07_berechnungen.ipynb)

## 🎯 Mathematische Grundlagen der linearen Regression

In dieser Vorlesung behandeln wir:
- ✅ **Normalgleichungen** - Die analytische Lösung für lineare Regression
- ✅ **Least Squares Prinzip** - Minimierung der quadrierten Fehler
- ✅ **Matrixform** - Elegante Darstellung mit $\boldsymbol{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$
- ✅ **MSE-Berechnung** - Mean Squared Error als Gütemaß
- ✅ **Praktische Anwendung** - Schritt-für-Schritt Berechnungen

**Warum ist das wichtig?** 🤔
- 🔬 **Grundlage für Machine Learning** - Basis für komplexere Algorithmen
- 📊 **Datenanalyse** - Trends und Zusammenhänge erkennen
- 🎯 **Vorhersagen** - Zukünftige Werte schätzen
- 🧮 **Mathematisches Verständnis** - Wie Computer "lernen"

## 🔧 Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns

# Für schöne Plots
%matplotlib inline
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("🎉 Alle Libraries geladen!")
print("📊 Bereit für mathematische Berechnungen und Visualisierungen!")

## 📊 Das Beispiel aus der Vorlesung

**Kühlanlage-Temperatur-Beispiel**: Energieverbrauch einer Kühlanlage bei verschiedenen Außentemperaturen

**Gegeben**: Drei Messpunkte
- Punkt 1: (10°C, 14 kWh/h)
- Punkt 2: (20°C, 19 kWh/h) 
- Punkt 3: (30°C, 25 kWh/h)

**Gesucht**: Beste Gerade $y = mx + b$ durch diese Punkte mittels **Normalgleichungen**

In [ ]:
# Die Datenpunkte aus der Vorlesung (exakt wie im Skript)
x_data = np.array([10, 20, 30])  # Außentemperatur in °C
y_data = np.array([14, 19, 25])  # Energieverbrauch in kWh/h

print("📋 Unsere Datenpunkte (aus Vorlesung 07):")
print("=" * 50)
print("Temperatur (°C) | Energieverbrauch (kWh/h)")
print("----------------|------------------------")
for i in range(len(x_data)):
    print(f"      {x_data[i]:2d}        |          {y_data[i]:2d}")

# Erste Visualisierung der Datenpunkte
plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, color='red', s=150, zorder=5, 
           edgecolors='black', linewidth=2, alpha=0.8)

# Punkte beschriften
for i, (x, y) in enumerate(zip(x_data, y_data)):
    plt.annotate(f'P{i+1}({x}°C, {y} kWh/h)', 
                (x, y), xytext=(10, 10), textcoords='offset points',
                fontweight='bold', fontsize=11,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

plt.xlabel('Außentemperatur (°C)', fontweight='bold')
plt.ylabel('Energieverbrauch Kühlung (kWh/h)', fontweight='bold')
plt.title('Kühlanlage: Energieverbrauch vs. Außentemperatur', fontweight='bold', fontsize=14)
plt.xlim(5, 35)
plt.ylim(10, 30)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🎯 Ziel: Finde die beste Gerade y = mx + b durch diese Punkte!")
print("📚 Methode: Normalgleichungen (Least Squares)")

## 🧮 Schritt 1: Grundwerte berechnen

Für die Normalgleichungen benötigen wir folgende Summen:

$$\sum_{i=1}^{n} x_i, \quad \sum_{i=1}^{n} y_i, \quad \sum_{i=1}^{n} x_i y_i, \quad \sum_{i=1}^{n} x_i^2$$

**Diese Werte sind die Basis für alle weiteren Berechnungen!**

In [ ]:
# Schritt 1: Grundwerte berechnen (wie im Skript)
n = len(x_data)
sum_x = np.sum(x_data)      # Σx_i
sum_y = np.sum(y_data)      # Σy_i  
sum_xy = np.sum(x_data * y_data)  # Σx_i*y_i
sum_x2 = np.sum(x_data**2)  # Σx_i²

print("📊 GRUNDWERTE für Normalgleichungen:")
print("=" * 50)
print(f"Anzahl Datenpunkte:     n = {n}")
print(f"Summe x-Werte:         Σx_i = {sum_x}")
print(f"Summe y-Werte:         Σy_i = {sum_y}")
print(f"Summe x*y-Produkte:  Σx_i*y_i = {sum_xy}")
print(f"Summe x²-Werte:       Σx_i² = {sum_x2}")

# Detaillierte Berechnung zeigen
print("\n🔍 DETAILLIERTE BERECHNUNG:")
print("=" * 50)
print("  i  |  x_i  |  y_i  | x_i*y_i | x_i²")
print("-----|-------|-------|---------|------")
total_xy = 0
total_x2 = 0
for i in range(n):
    xy_product = x_data[i] * y_data[i]
    x_squared = x_data[i]**2
    total_xy += xy_product
    total_x2 += x_squared
    print(f"  {i+1}  |   {x_data[i]:2d}  |   {y_data[i]:2d}  |   {xy_product:3d}   |  {x_squared:3d}")

print("-----|-------|-------|---------|------")
print(f" Σ   |   {sum_x:2d}  |   {sum_y:2d}  |   {total_xy:3d}   | {total_x2:4d}")

# Kontrolle
print(f"\n✅ KONTROLLE:")
print(f"   Σx_i*y_i berechnet: {total_xy} = {sum_xy} ✓")
print(f"   Σx_i² berechnet: {total_x2} = {sum_x2} ✓")

## 🎯 Schritt 2: Normalgleichungen anwenden

Die **Normalgleichungen** für lineare Regression lauten:

$$m = \frac{n \sum x_i y_i - \sum x_i \sum y_i}{n \sum x_i^2 - (\sum x_i)^2}$$

$$b = \frac{\sum y_i - m \sum x_i}{n}$$

**Diese Formeln minimieren automatisch den MSE (Mean Squared Error)!**

In [ ]:
print("🎯 NORMALGLEICHUNGEN SCHRITT FÜR SCHRITT:")
print("=" * 60)

# Schritt 2a: Steigung m berechnen
print("📐 STEIGUNG m berechnen:")
print(f"   m = (n×Σx_i*y_i - Σx_i×Σy_i) / (n×Σx_i² - (Σx_i)²)")

# Zähler berechnen
numerator_m = n * sum_xy - sum_x * sum_y
print(f"   Zähler = {n}×{sum_xy} - {sum_x}×{sum_y}")
print(f"   Zähler = {n * sum_xy} - {sum_x * sum_y} = {numerator_m}")

# Nenner berechnen
denominator_m = n * sum_x2 - sum_x**2
print(f"   Nenner = {n}×{sum_x2} - {sum_x}²")
print(f"   Nenner = {n * sum_x2} - {sum_x**2} = {denominator_m}")

# Steigung
m_analytical = numerator_m / denominator_m
print(f"   m = {numerator_m} / {denominator_m} = {m_analytical:.6f}")

print("\n📏 Y-ACHSENABSCHNITT b berechnen:")
print(f"   b = (Σy_i - m×Σx_i) / n")
print(f"   b = ({sum_y} - {m_analytical:.6f}×{sum_x}) / {n}")

# Y-Achsenabschnitt
b_analytical = (sum_y - m_analytical * sum_x) / n
print(f"   b = ({sum_y} - {m_analytical * sum_x:.3f}) / {n}")
print(f"   b = {sum_y - m_analytical * sum_x:.3f} / {n} = {b_analytical:.6f}")

print("\n🎉 ERGEBNIS:")
print("=" * 30)
print(f"📐 Steigung:           m = {m_analytical:.6f}")
print(f"📏 Y-Achsenabschnitt:  b = {b_analytical:.6f}")
print(f"📈 Gleichung:    y = {m_analytical:.3f}x + {b_analytical:.2f}")

# Physikalische Interpretation
print("\n🔬 PHYSIKALISCHE INTERPRETATION:")
print(f"   • Pro 1°C Temperaturanstieg steigt der Energieverbrauch um {m_analytical:.3f} kWh/h")
print(f"   • Bei 0°C würde die Anlage theoretisch {b_analytical:.1f} kWh/h verbrauchen")
print(f"   • Positive Steigung bestätigt: Wärmer → mehr Kühlenergie nötig")

## 📊 Schritt 3: MSE (Mean Squared Error) berechnen

Der **MSE** misst die Güte unserer Regression:

$$\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 = \frac{1}{n} \sum_{i=1}^{n} (y_i - (mx_i + b))^2$$

**Je kleiner der MSE, desto besser die Anpassung!**

In [ ]:
print("📊 MSE (MEAN SQUARED ERROR) BERECHNUNG:")
print("=" * 60)

# MSE Schritt für Schritt berechnen
print("🔍 SCHRITT-FÜR-SCHRITT BERECHNUNG:")
print("  i  | x_i | y_i | ŷ_i = mx_i+b | Residuum | Residuum²")
print("-----|-----|-----|--------------|----------|----------")

total_squared_error = 0
residuals = []
predictions = []

for i in range(n):
    x_i = x_data[i]
    y_i = y_data[i]
    y_pred = m_analytical * x_i + b_analytical
    residual = y_i - y_pred
    squared_residual = residual**2
    
    predictions.append(y_pred)
    residuals.append(residual)
    total_squared_error += squared_residual
    
    print(f"  {i+1}  | {x_i:2d}  | {y_i:2d}  |    {y_pred:6.2f}    |  {residual:6.2f}  |  {squared_residual:6.4f}")

mse_analytical = total_squared_error / n

print("-----|-----|-----|--------------|----------|----------")
print(f" Σ   |     |     |              |          | {total_squared_error:7.4f}")
print(f"\n📈 MSE = Σ(Residuum²) / n = {total_squared_error:.4f} / {n} = {mse_analytical:.6f}")

# Zusätzliche Gütemaße
predictions = np.array(predictions)
residuals = np.array(residuals)

# R² (Bestimmtheitsgmaß) berechnen
ss_res = np.sum(residuals**2)  # Sum of squares of residuals
ss_tot = np.sum((y_data - np.mean(y_data))**2)  # Total sum of squares
r_squared = 1 - (ss_res / ss_tot)

# RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse_analytical)

print("\n📊 ZUSÄTZLICHE GÜTEMAßE:")
print("=" * 30)
print(f"📈 MSE  = {mse_analytical:.6f}")
print(f"📐 RMSE = {rmse:.6f} kWh/h")
print(f"📊 R²   = {r_squared:.6f} ({r_squared*100:.2f}%)")

print("\n💡 INTERPRETATION:")
print(f"   • MSE von {mse_analytical:.3f} bedeutet mittlerer quadrierter Fehler")
print(f"   • RMSE von {rmse:.3f} kWh/h ist der mittlere absolute Fehler")
print(f"   • R² von {r_squared:.3f} bedeutet {r_squared*100:.1f}% der Varianz wird erklärt")

if r_squared > 0.9:
    print("   🎉 Ausgezeichnete Anpassung!")
elif r_squared > 0.7:
    print("   ✅ Gute Anpassung!")
else:
    print("   ⚠️ Anpassung könnte besser sein")

## 📈 Schritt 4: Visualisierung der Regression

Jetzt visualisieren wir:
- ✅ **Datenpunkte** und **Regressionsgerade**
- ✅ **Residuen** (Abweichungen)
- ✅ **Vorhersagen** für neue Werte
- ✅ **Konfidenzintervalle**

In [ ]:
# Große Visualisierung der Regression
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Regression mit Datenpunkten
x_line = np.linspace(0, 40, 100)
y_line = m_analytical * x_line + b_analytical

ax1.scatter(x_data, y_data, color='red', s=150, zorder=5, 
           edgecolors='black', linewidth=2, label='Datenpunkte')
ax1.plot(x_line, y_line, 'b-', linewidth=3, alpha=0.8, 
         label=f'Regression: y = {m_analytical:.3f}x + {b_analytical:.2f}')

# Vorhersagen für die Datenpunkte markieren
ax1.scatter(x_data, predictions, color='blue', s=100, alpha=0.7, 
           marker='x', linewidth=3, label='Vorhersagen')

# Residuen als Linien
for i in range(n):
    ax1.plot([x_data[i], x_data[i]], [y_data[i], predictions[i]], 
            'gray', linestyle='--', alpha=0.7, linewidth=2)

ax1.set_xlabel('Außentemperatur (°C)', fontweight='bold')
ax1.set_ylabel('Energieverbrauch (kWh/h)', fontweight='bold')
ax1.set_title('Lineare Regression - Hauptdiagramm', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 40)
ax1.set_ylim(10, 30)

# Plot 2: Residuen-Plot
ax2.scatter(x_data, residuals, color='purple', s=120, alpha=0.8)
ax2.axhline(y=0, color='red', linestyle='-', alpha=0.7, linewidth=2)
ax2.set_xlabel('Außentemperatur (°C)', fontweight='bold')
ax2.set_ylabel('Residuen (kWh/h)', fontweight='bold')
ax2.set_title('Residuen-Plot', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Residuen beschriften
for i, (x, res) in enumerate(zip(x_data, residuals)):
    ax2.annotate(f'{res:.2f}', (x, res), xytext=(5, 5), 
                textcoords='offset points', fontweight='bold')

# Plot 3: Q-Q Plot der Residuen (Normalverteilung prüfen)
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normalverteilung der Residuen)', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Vorhersagen für erweiterten Bereich
x_extended = np.linspace(-5, 45, 200)
y_extended = m_analytical * x_extended + b_analytical

ax4.plot(x_extended, y_extended, 'b-', linewidth=3, alpha=0.8, 
         label=f'Regression (erweitert)')
ax4.scatter(x_data, y_data, color='red', s=150, zorder=5, 
           edgecolors='black', linewidth=2, label='Datenpunkte')

# Interessante Vorhersagen markieren
temp_predictions = [0, 25, 35, 40]
for temp in temp_predictions:
    energy = m_analytical * temp + b_analytical
    ax4.plot(temp, energy, 'go', markersize=10, alpha=0.8)
    ax4.annotate(f'({temp}°C, {energy:.1f} kWh/h)', 
                (temp, energy), xytext=(5, 5), textcoords='offset points',
                fontweight='bold', fontsize=9,
                bbox=dict(boxstyle='round,pad=0.2', facecolor='lightgreen', alpha=0.7))

ax4.set_xlabel('Außentemperatur (°C)', fontweight='bold')
ax4.set_ylabel('Energieverbrauch (kWh/h)', fontweight='bold')
ax4.set_title('Vorhersagen für erweiterten Temperaturbereich', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_xlim(-5, 45)
ax4.set_ylim(5, 35)

plt.tight_layout()
plt.show()

print("📊 VISUALISIERUNG KOMPLETT!")
print("=" * 40)
print("🔍 Oben links:  Hauptregression mit Residuen")
print("🔍 Oben rechts: Residuen-Plot (sollten zufällig um 0 streuen)")
print("🔍 Unten links: Q-Q Plot (prüft Normalverteilung der Residuen)")
print("🔍 Unten rechts: Vorhersagen für erweiterten Bereich")

## 🔬 Schritt 5: Vergleich mit anderen Methoden

Wir vergleichen unsere **handberechneten Normalgleichungen** mit:
- ✅ **NumPy polyfit** - Polynomiale Anpassung
- ✅ **SciPy linregress** - Statistische lineare Regression
- ✅ **Scikit-learn** - Machine Learning Ansatz

In [ ]:
print("🔬 VERGLEICH MIT ANDEREN METHODEN:")
print("=" * 60)

# Methode 1: NumPy polyfit
poly_coeff = np.polyfit(x_data, y_data, 1)
m_numpy, b_numpy = poly_coeff[0], poly_coeff[1]
mse_numpy = np.mean((y_data - (m_numpy * x_data + b_numpy))**2)

print("1️⃣ NumPy polyfit:")
print(f"   m = {m_numpy:.6f}, b = {b_numpy:.6f}, MSE = {mse_numpy:.8f}")

# Methode 2: SciPy linregress
slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, y_data)
mse_scipy = np.mean((y_data - (slope * x_data + intercept))**2)

print("\n2️⃣ SciPy linregress:")
print(f"   m = {slope:.6f}, b = {intercept:.6f}, MSE = {mse_scipy:.8f}")
print(f"   R = {r_value:.6f}, R² = {r_value**2:.6f}")
print(f"   p-value = {p_value:.8f}, std_err = {std_err:.8f}")

# Methode 3: Scikit-learn (falls verfügbar)
try:
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, r2_score
    
    # Scikit-learn erwartet 2D Arrays
    X_sklearn = x_data.reshape(-1, 1)
    
    model = LinearRegression()
    model.fit(X_sklearn, y_data)
    
    m_sklearn = model.coef_[0]
    b_sklearn = model.intercept_
    y_pred_sklearn = model.predict(X_sklearn)
    mse_sklearn = mean_squared_error(y_data, y_pred_sklearn)
    r2_sklearn = r2_score(y_data, y_pred_sklearn)
    
    print("\n3️⃣ Scikit-learn LinearRegression:")
    print(f"   m = {m_sklearn:.6f}, b = {b_sklearn:.6f}, MSE = {mse_sklearn:.8f}")
    print(f"   R² = {r2_sklearn:.6f}")
    
except ImportError:
    print("\n3️⃣ Scikit-learn: Nicht verfügbar (pip install scikit-learn)")
    m_sklearn, b_sklearn, mse_sklearn = m_analytical, b_analytical, mse_analytical

# Vergleichstabelle
print("\n📊 VERGLEICHSTABELLE:")
print("=" * 70)
print(f"{'Methode':<25} {'Steigung m':<12} {'Y-Abschn. b':<12} {'MSE':<12}")
print("-" * 70)
print(f"{'Normalgleichungen':<25} {m_analytical:<12.6f} {b_analytical:<12.6f} {mse_analytical:<12.8f}")
print(f"{'NumPy polyfit':<25} {m_numpy:<12.6f} {b_numpy:<12.6f} {mse_numpy:<12.8f}")
print(f"{'SciPy linregress':<25} {slope:<12.6f} {intercept:<12.6f} {mse_scipy:<12.8f}")
try:
    print(f"{'Scikit-learn':<25} {m_sklearn:<12.6f} {b_sklearn:<12.6f} {mse_sklearn:<12.8f}")
except:
    pass

# Differenzen berechnen
diff_m_numpy = abs(m_analytical - m_numpy)
diff_b_numpy = abs(b_analytical - b_numpy)
diff_m_scipy = abs(m_analytical - slope)
diff_b_scipy = abs(b_analytical - intercept)

print("\n✅ ÜBEREINSTIMMUNG (Abweichungen):")
print(f"   NumPy:     Δm = {diff_m_numpy:.10f}, Δb = {diff_b_numpy:.10f}")
print(f"   SciPy:     Δm = {diff_m_scipy:.10f}, Δb = {diff_b_scipy:.10f}")

if diff_m_numpy < 1e-10 and diff_b_numpy < 1e-10:
    print("\n🎉 PERFEKT! Alle Methoden liefern identische Ergebnisse!")
    print("💡 Das bestätigt unsere Normalgleichungen-Implementierung!")
else:
    print("\n⚠️ Kleine numerische Unterschiede (normal bei Floating-Point-Arithmetik)")

## 🎯 Schritt 6: Praktische Anwendungen

**Jetzt können wir Vorhersagen machen!**

Unsere Gleichung: $y = 0.550x + 8.500$

Was bedeutet das für die Praxis?

In [ ]:
print("🎯 PRAKTISCHE ANWENDUNGEN:")
print("=" * 50)

# Regressionsfunktion definieren
def predict_energy(temperature):
    """Vorhersage des Energieverbrauchs basierend auf Temperatur"""
    return m_analytical * temperature + b_analytical

# Interessante Vorhersagen
scenarios = [
    ("Kalter Winter", 0),
    ("Milder Frühling", 15),
    ("Warmer Sommer", 35),
    ("Hitzewelle", 40),
    ("Extremhitze", 45)
]

print("🌡️ ENERGIEVERBRAUCH-VORHERSAGEN:")
print("-" * 50)
print(f"{'Szenario':<15} {'Temp (°C)':<10} {'Energie (kWh/h)':<15} {'Kommentar'}")
print("-" * 50)

for scenario, temp in scenarios:
    energy = predict_energy(temp)
    
    if energy < 15:
        comment = "Niedriger Verbrauch"
    elif energy < 25:
        comment = "Moderater Verbrauch"
    else:
        comment = "Hoher Verbrauch"
    
    print(f"{scenario:<15} {temp:<10} {energy:<15.1f} {comment}")

# Kostenberechnung (Beispiel)
energy_cost_per_kwh = 0.30  # 30 Cent pro kWh

print(f"\n💰 KOSTENBERECHNUNG (bei {energy_cost_per_kwh:.2f} €/kWh):")
print("-" * 60)
print(f"{'Szenario':<15} {'Temp (°C)':<10} {'Kosten/Stunde (€)':<18} {'Kosten/Tag (€)'}")
print("-" * 60)

for scenario, temp in scenarios:
    energy = predict_energy(temp)
    cost_per_hour = energy * energy_cost_per_kwh
    cost_per_day = cost_per_hour * 24
    
    print(f"{scenario:<15} {temp:<10} {cost_per_hour:<18.2f} {cost_per_day:<12.2f}")

# Energieeinsparung durch Temperaturreduktion
print(f"\n🌱 ENERGIEEINSPARUNG durch 1°C Temperaturreduktion:")
print(f"   • Einsparung: {m_analytical:.3f} kWh/h")
print(f"   • Kosteneinsparung: {m_analytical * energy_cost_per_kwh:.3f} €/Stunde")
print(f"   • Kosteneinsparung: {m_analytical * energy_cost_per_kwh * 24:.2f} €/Tag")
print(f"   • Kosteneinsparung: {m_analytical * energy_cost_per_kwh * 24 * 365:.0f} €/Jahr")

print(f"\n💡 PRAKTISCHE ERKENNTNISSE:")
print(f"   • Bei 0°C: Grundverbrauch von {b_analytical:.1f} kWh/h (Ventilation, etc.)")
print(f"   • Linearer Anstieg: {m_analytical:.3f} kWh/h pro °C")
print(f"   • Energieeffizienz: Temperatur um nur 1°C senken spart deutlich Kosten!")
print(f"   • Planbarkeit: Energiebedarf bei jeder Temperatur vorhersagbar")

## 🧮 Schritt 7: Matrixform der Normalgleichungen

**Die elegante Matrixschreibweise:**

$$\boldsymbol{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

wo:
- $\boldsymbol{\beta} = \begin{pmatrix} b \\ m \end{pmatrix}$ (Parameter)
- $\mathbf{X} = \begin{pmatrix} 1 & x_1 \\ 1 & x_2 \\ 1 & x_3 \end{pmatrix}$ (Design-Matrix)
- $\mathbf{y} = \begin{pmatrix} y_1 \\ y_2 \\ y_3 \end{pmatrix}$ (Zielwerte)

In [ ]:
print("🧮 MATRIXFORM DER NORMALGLEICHUNGEN:")
print("=" * 60)

# Design-Matrix X erstellen (erste Spalte: 1en, zweite Spalte: x-Werte)
X = np.column_stack([np.ones(len(x_data)), x_data])
y = y_data.reshape(-1, 1)

print("📊 MATRIZEN AUFSTELLEN:")
print(f"\nDesign-Matrix X (n×2):")
print(X)
print(f"\nZielvektor y (n×1):")
print(y.flatten())

# Schritt 1: X^T berechnen
X_transpose = X.T
print(f"\n🔄 X^T (Transponierte):")
print(X_transpose)

# Schritt 2: X^T * X berechnen
XTX = X_transpose @ X
print(f"\n🔢 X^T * X (2×2 Matrix):")
print(XTX)
print(f"\nInterpretation:")
print(f"   XTX[0,0] = {XTX[0,0]} = n = Anzahl Datenpunkte")
print(f"   XTX[0,1] = {XTX[0,1]} = Σx_i = Summe der x-Werte")
print(f"   XTX[1,0] = {XTX[1,0]} = Σx_i = Summe der x-Werte")
print(f"   XTX[1,1] = {XTX[1,1]} = Σx_i² = Summe der quadrierten x-Werte")

# Schritt 3: (X^T * X)^(-1) berechnen
XTX_inv = np.linalg.inv(XTX)
print(f"\n🔄 (X^T * X)^(-1) (Inverse):")
print(XTX_inv)

# Schritt 4: X^T * y berechnen
XTy = X_transpose @ y
print(f"\n🔢 X^T * y (2×1 Vektor):")
print(XTy.flatten())
print(f"\nInterpretation:")
print(f"   XTy[0] = {XTy[0,0]} = Σy_i = Summe der y-Werte")
print(f"   XTy[1] = {XTy[1,0]} = Σx_i*y_i = Summe der Produkte")

# Schritt 5: β = (X^T * X)^(-1) * X^T * y
beta = XTX_inv @ XTy
b_matrix, m_matrix = beta[0,0], beta[1,0]

print(f"\n🎯 ENDERGEBNIS β = (X^T * X)^(-1) * X^T * y:")
print(f"β = {beta.flatten()}")
print(f"\nParameter:")
print(f"   b (Y-Achsenabschnitt) = {b_matrix:.6f}")
print(f"   m (Steigung)          = {m_matrix:.6f}")

# Vergleich mit unserer direkten Berechnung
print(f"\n✅ VERGLEICH MIT DIREKTER BERECHNUNG:")
print(f"{'Methode':<20} {'Steigung m':<15} {'Y-Abschnitt b':<15}")
print("-" * 50)
print(f"{'Normalgleichungen':<20} {m_analytical:<15.6f} {b_analytical:<15.6f}")
print(f"{'Matrixform':<20} {m_matrix:<15.6f} {b_matrix:<15.6f}")

diff_m_matrix = abs(m_analytical - m_matrix)
diff_b_matrix = abs(b_analytical - b_matrix)
print(f"\nAbweichungen: Δm = {diff_m_matrix:.10f}, Δb = {diff_b_matrix:.10f}")

if diff_m_matrix < 1e-10 and diff_b_matrix < 1e-10:
    print("🎉 IDENTISCHE ERGEBNISSE! Matrixform bestätigt Normalgleichungen!")
else:
    print("⚠️ Kleine numerische Unterschiede")

print(f"\n💡 WARUM MATRIXFORM?")
print(f"   • Elegante mathematische Notation")
print(f"   • Einfach auf mehr Variablen erweiterbar")
print(f"   • Effiziente Computerimplementierung")
print(f"   • Basis für komplexere ML-Algorithmen")

## 📊 Zusammenfassung: Was haben wir gelernt?

### 🎯 **Kernkonzepte**:
1. **Normalgleichungen** - Analytische Lösung für lineare Regression
2. **Least Squares** - Minimierung der quadrierten Fehler
3. **MSE** - Quantifizierung der Modellgüte
4. **Matrixform** - Elegante mathematische Darstellung

### 🔬 **Praktische Erkenntnisse**:
- Energieverbrauch steigt linear mit der Temperatur
- Vorhersagen für beliebige Temperaturen möglich
- Kosteneinsparungen durch Temperaturoptimierung quantifizierbar

### 💡 **Warum ist das wichtig für Data Science?**
- **Basis für Machine Learning** - Verständnis der Grundprinzipien
- **Interpretierbarkeit** - Klare mathematische Beziehungen
- **Effizienz** - Schnelle analytische Lösung
- **Erweiterbarkeit** - Grundlage für komplexere Modelle

In [ ]:
# Abschließende Zusammenfassung mit allen wichtigen Ergebnissen
print("📋 VOLLSTÄNDIGE ZUSAMMENFASSUNG - VORLESUNG 07")
print("=" * 70)

print(f"📊 DATENBASIS:")
print(f"   • {n} Datenpunkte: Temperatur vs. Energieverbrauch")
print(f"   • Bereich: {x_data.min()}°C bis {x_data.max()}°C")
print(f"   • Energiebereich: {y_data.min()} bis {y_data.max()} kWh/h")

print(f"\n🎯 REGRESSIONSERGEBNIS:")
print(f"   • Gleichung: y = {m_analytical:.3f}x + {b_analytical:.2f}")
print(f"   • Steigung: {m_analytical:.3f} kWh/h pro °C")
print(f"   • Y-Achsenabschnitt: {b_analytical:.2f} kWh/h bei 0°C")

print(f"\n📈 MODELLGÜTE:")
print(f"   • MSE: {mse_analytical:.6f}")
print(f"   • RMSE: {np.sqrt(mse_analytical):.3f} kWh/h")
print(f"   • R²: {r_squared:.3f} ({r_squared*100:.1f}% Varianz erklärt)")

print(f"\n💰 PRAKTISCHE ANWENDUNG:")
energy_at_20 = predict_energy(20)
energy_at_30 = predict_energy(30)
print(f"   • Bei 20°C: {energy_at_20:.1f} kWh/h")
print(f"   • Bei 30°C: {energy_at_30:.1f} kWh/h")
print(f"   • Einsparung pro °C: {m_analytical:.3f} kWh/h")
daily_savings = m_analytical * 24 * energy_cost_per_kwh
print(f"   • Kosteneinsparung pro °C: {daily_savings:.2f} €/Tag")

print(f"\n✅ VALIDIERUNG:")
print(f"   • Normalgleichungen ≡ NumPy ≡ SciPy ≡ Matrixform")
print(f"   • Alle Methoden liefern identische Ergebnisse")
print(f"   • Residuen sind klein und zufällig verteilt")

print(f"\n🎓 LERNERFOLG:")
print(f"   ✅ Normalgleichungen verstanden und angewendet")
print(f"   ✅ MSE-Minimierung nachvollzogen")
print(f"   ✅ Matrixform der linearen Regression beherrscht")
print(f"   ✅ Praktische Anwendung und Interpretation gelernt")
print(f"   ✅ Grundlage für erweiterte ML-Algorithmen geschaffen")

print(f"\n🚀 NÄCHSTE SCHRITTE:")
print(f"   • Multiple lineare Regression (mehrere Variablen)")
print(f"   • Nichtlineare Regression")
print(f"   • Regularisierung (Ridge, Lasso)")
print(f"   • Neuronale Netze als Erweiterung")

print(f"\n" + "=" * 70)
print(f"🎉 VORLESUNG 07 ERFOLGREICH ABGESCHLOSSEN! 🎉")
print(f"=" * 70)